<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Regression Models with Keras</font></h1>


## Introduction


As we discussed in the videos, despite the popularity of more powerful libraries such as PyToch and TensorFlow, they are not easy to use and have a steep learning curve. So, for people who are just starting to learn deep learning, there is no better library to use other than the Keras library. 

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

So, in this lab, you will learn how to use the Keras library to build a regression model.


<h2>Regression Models with Keras</h2>

<h3>Objective for this Notebook<h3>    
<h5> 1. How to use the Keras library to build a regression model.</h5>
<h5> 2. Download and Clean dataset </h5>
<h5> 3. Build a Neural Network </h5>
<h5> 4. Train and Test the Network. </h5>     


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Download and Clean Dataset</a>  
2. <a href="#item32">Import Keras</a>  
3. <a href="#item33">Build a Neural Network</a>  
4. <a href="#item34">Train and Test the Network</a>  

</font>
</div>


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [9]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's save the number of predictors to _n_cols_ since we will need this number when building our network.


In [10]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.


#### Let's go ahead and import the Keras library


In [15]:
!pip install tensorflow keras
import keras

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 320.5 MB 5.6 kB/s ta 0:00:011    |███                             | 29.7 MB 790 kB/s eta 0:06:08     |████                            | 39.5 MB 311 kB/s eta 0:15:04     |██████▎                         | 62.5 MB 69 kB/s eta 1:01:25     |████████████████                | 160.2 MB 84 kB/s eta 0:31:42     |█████████████████████▏          | 212.5 MB 28 kB/s eta 1:03:33     |█████████████████████▎          | 213.3 MB 48 kB/s eta 0:36:42     |█████████████████████▍          | 214.3 MB 39 kB/s eta 0:44:56     |█████████████████████▊          | 217.3 MB 37 kB/s eta 0:45:47     |█████████████████████▉          | 218.8 MB 280 kB/s eta 0:06:03     |██████████████████████▌         | 225.2 MB 202 kB/s eta 0:07:51     |███████████████████████▋        | 236.6 MB 39 kB/s eta 0:35:46     |████████████████████████▍       | 244.6 MB 36 kB/s eta 0:34:14     |███████████████████████████▊   

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [16]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [20]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has two hidden layers, each of 50 hidden units.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [21]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model for 100 epochs.


In [23]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=1000, verbose=2)

Epoch 1/1000
23/23 - 0s - loss: 1644.0314 - val_loss: 1086.6315
Epoch 2/1000
23/23 - 0s - loss: 1053.4258 - val_loss: 229.2384
Epoch 3/1000
23/23 - 0s - loss: 307.0479 - val_loss: 226.9986
Epoch 4/1000
23/23 - 0s - loss: 232.5377 - val_loss: 197.6065
Epoch 5/1000
23/23 - 0s - loss: 210.4880 - val_loss: 188.3550
Epoch 6/1000
23/23 - 0s - loss: 191.7186 - val_loss: 181.2168
Epoch 7/1000
23/23 - 0s - loss: 179.5155 - val_loss: 175.8161
Epoch 8/1000
23/23 - 0s - loss: 167.7006 - val_loss: 171.7542
Epoch 9/1000
23/23 - 0s - loss: 156.7281 - val_loss: 168.8986
Epoch 10/1000
23/23 - 0s - loss: 143.6813 - val_loss: 155.5691
Epoch 11/1000
23/23 - 0s - loss: 132.3729 - val_loss: 154.5312
Epoch 12/1000
23/23 - 0s - loss: 119.3645 - val_loss: 155.5618
Epoch 13/1000
23/23 - 0s - loss: 107.0916 - val_loss: 161.9939
Epoch 14/1000
23/23 - 0s - loss: 98.3260 - val_loss: 153.7753
Epoch 15/1000
23/23 - 0s - loss: 88.5603 - val_loss: 156.9787
Epoch 16/1000
23/23 - 0s - loss: 79.3052 - val_loss: 164.4285
E

Epoch 133/1000
23/23 - 0s - loss: 8.1594 - val_loss: 173.6785
Epoch 134/1000
23/23 - 0s - loss: 8.6509 - val_loss: 189.4706
Epoch 135/1000
23/23 - 0s - loss: 8.1835 - val_loss: 172.7440
Epoch 136/1000
23/23 - 0s - loss: 8.4585 - val_loss: 197.7529
Epoch 137/1000
23/23 - 0s - loss: 8.6539 - val_loss: 176.3097
Epoch 138/1000
23/23 - 0s - loss: 8.5127 - val_loss: 183.1072
Epoch 139/1000
23/23 - 0s - loss: 8.4538 - val_loss: 191.5248
Epoch 140/1000
23/23 - 0s - loss: 8.8108 - val_loss: 179.8608
Epoch 141/1000
23/23 - 0s - loss: 10.5173 - val_loss: 180.7947
Epoch 142/1000
23/23 - 0s - loss: 8.3646 - val_loss: 189.5302
Epoch 143/1000
23/23 - 0s - loss: 7.7913 - val_loss: 175.5508
Epoch 144/1000
23/23 - 0s - loss: 8.1961 - val_loss: 196.5252
Epoch 145/1000
23/23 - 0s - loss: 8.2773 - val_loss: 182.9440
Epoch 146/1000
23/23 - 0s - loss: 7.7993 - val_loss: 182.0970
Epoch 147/1000
23/23 - 0s - loss: 8.6632 - val_loss: 213.7310
Epoch 148/1000
23/23 - 0s - loss: 9.0333 - val_loss: 168.5151
Epoch 1

23/23 - 0s - loss: 5.6868 - val_loss: 188.6404
Epoch 266/1000
23/23 - 0s - loss: 6.7287 - val_loss: 226.3711
Epoch 267/1000
23/23 - 0s - loss: 7.2555 - val_loss: 215.9179
Epoch 268/1000
23/23 - 0s - loss: 7.1770 - val_loss: 216.7956
Epoch 269/1000
23/23 - 0s - loss: 5.8839 - val_loss: 211.2960
Epoch 270/1000
23/23 - 0s - loss: 5.4171 - val_loss: 197.6464
Epoch 271/1000
23/23 - 0s - loss: 6.5788 - val_loss: 196.3097
Epoch 272/1000
23/23 - 0s - loss: 6.5648 - val_loss: 222.6470
Epoch 273/1000
23/23 - 0s - loss: 5.8047 - val_loss: 200.4686
Epoch 274/1000
23/23 - 0s - loss: 5.8747 - val_loss: 217.6871
Epoch 275/1000
23/23 - 0s - loss: 5.5220 - val_loss: 206.8847
Epoch 276/1000
23/23 - 0s - loss: 7.9782 - val_loss: 192.8548
Epoch 277/1000
23/23 - 0s - loss: 8.6828 - val_loss: 195.5553
Epoch 278/1000
23/23 - 0s - loss: 8.1105 - val_loss: 194.5507
Epoch 279/1000
23/23 - 0s - loss: 7.6644 - val_loss: 212.4801
Epoch 280/1000
23/23 - 0s - loss: 6.0467 - val_loss: 215.9021
Epoch 281/1000
23/23 - 

Epoch 398/1000
23/23 - 0s - loss: 5.7144 - val_loss: 229.0659
Epoch 399/1000
23/23 - 0s - loss: 4.8634 - val_loss: 209.3077
Epoch 400/1000
23/23 - 0s - loss: 5.5863 - val_loss: 215.3094
Epoch 401/1000
23/23 - 0s - loss: 5.3927 - val_loss: 241.4941
Epoch 402/1000
23/23 - 0s - loss: 7.3448 - val_loss: 224.8668
Epoch 403/1000
23/23 - 0s - loss: 6.7231 - val_loss: 258.2526
Epoch 404/1000
23/23 - 0s - loss: 5.6090 - val_loss: 238.9551
Epoch 405/1000
23/23 - 0s - loss: 4.8804 - val_loss: 217.5961
Epoch 406/1000
23/23 - 0s - loss: 5.0711 - val_loss: 220.9106
Epoch 407/1000
23/23 - 0s - loss: 4.7812 - val_loss: 217.0637
Epoch 408/1000
23/23 - 0s - loss: 4.1030 - val_loss: 224.3864
Epoch 409/1000
23/23 - 0s - loss: 4.4490 - val_loss: 209.2106
Epoch 410/1000
23/23 - 0s - loss: 5.6569 - val_loss: 213.6442
Epoch 411/1000
23/23 - 0s - loss: 5.5756 - val_loss: 238.7373
Epoch 412/1000
23/23 - 0s - loss: 6.5071 - val_loss: 200.1343
Epoch 413/1000
23/23 - 0s - loss: 5.7485 - val_loss: 240.7993
Epoch 41

23/23 - 0s - loss: 5.8121 - val_loss: 210.1161
Epoch 531/1000
23/23 - 0s - loss: 4.9995 - val_loss: 217.0689
Epoch 532/1000
23/23 - 0s - loss: 4.6625 - val_loss: 217.9614
Epoch 533/1000
23/23 - 0s - loss: 7.1490 - val_loss: 235.1091
Epoch 534/1000
23/23 - 0s - loss: 7.5517 - val_loss: 210.6635
Epoch 535/1000
23/23 - 0s - loss: 6.5522 - val_loss: 234.6982
Epoch 536/1000
23/23 - 0s - loss: 5.0903 - val_loss: 221.1869
Epoch 537/1000
23/23 - 0s - loss: 4.7477 - val_loss: 208.6183
Epoch 538/1000
23/23 - 0s - loss: 4.6309 - val_loss: 226.5966
Epoch 539/1000
23/23 - 0s - loss: 4.8531 - val_loss: 210.6314
Epoch 540/1000
23/23 - 0s - loss: 4.5234 - val_loss: 231.7226
Epoch 541/1000
23/23 - 0s - loss: 4.3364 - val_loss: 213.3717
Epoch 542/1000
23/23 - 0s - loss: 4.5065 - val_loss: 231.9890
Epoch 543/1000
23/23 - 0s - loss: 4.0953 - val_loss: 215.6546
Epoch 544/1000
23/23 - 0s - loss: 4.2345 - val_loss: 213.0702
Epoch 545/1000
23/23 - 0s - loss: 4.5939 - val_loss: 212.0974
Epoch 546/1000
23/23 - 

Epoch 663/1000
23/23 - 0s - loss: 4.1928 - val_loss: 214.4362
Epoch 664/1000
23/23 - 0s - loss: 4.2412 - val_loss: 221.7344
Epoch 665/1000
23/23 - 0s - loss: 3.8187 - val_loss: 208.3822
Epoch 666/1000
23/23 - 0s - loss: 3.8105 - val_loss: 226.2867
Epoch 667/1000
23/23 - 0s - loss: 3.6907 - val_loss: 210.5976
Epoch 668/1000
23/23 - 0s - loss: 3.8449 - val_loss: 222.3726
Epoch 669/1000
23/23 - 0s - loss: 5.0342 - val_loss: 216.1232
Epoch 670/1000
23/23 - 0s - loss: 6.0981 - val_loss: 219.9903
Epoch 671/1000
23/23 - 0s - loss: 6.5091 - val_loss: 234.9605
Epoch 672/1000
23/23 - 0s - loss: 6.5044 - val_loss: 201.8294
Epoch 673/1000
23/23 - 0s - loss: 5.0587 - val_loss: 212.7557
Epoch 674/1000
23/23 - 0s - loss: 3.9521 - val_loss: 211.5347
Epoch 675/1000
23/23 - 0s - loss: 3.8568 - val_loss: 221.8833
Epoch 676/1000
23/23 - 0s - loss: 3.8461 - val_loss: 210.1675
Epoch 677/1000
23/23 - 0s - loss: 3.6079 - val_loss: 206.7438
Epoch 678/1000
23/23 - 0s - loss: 3.8757 - val_loss: 222.8763
Epoch 67

23/23 - 0s - loss: 3.2553 - val_loss: 210.0693
Epoch 796/1000
23/23 - 0s - loss: 3.8717 - val_loss: 206.5385
Epoch 797/1000
23/23 - 0s - loss: 3.6113 - val_loss: 211.8749
Epoch 798/1000
23/23 - 0s - loss: 4.2681 - val_loss: 203.9681
Epoch 799/1000
23/23 - 0s - loss: 4.0810 - val_loss: 210.7664
Epoch 800/1000
23/23 - 0s - loss: 3.8445 - val_loss: 207.4502
Epoch 801/1000
23/23 - 0s - loss: 4.0394 - val_loss: 223.7930
Epoch 802/1000
23/23 - 0s - loss: 4.2140 - val_loss: 219.8142
Epoch 803/1000
23/23 - 0s - loss: 3.9597 - val_loss: 234.0184
Epoch 804/1000
23/23 - 0s - loss: 4.2017 - val_loss: 211.4677
Epoch 805/1000
23/23 - 0s - loss: 5.8700 - val_loss: 236.5916
Epoch 806/1000
23/23 - 0s - loss: 4.2729 - val_loss: 197.5203
Epoch 807/1000
23/23 - 0s - loss: 4.9729 - val_loss: 209.6149
Epoch 808/1000
23/23 - 0s - loss: 4.3302 - val_loss: 216.3695
Epoch 809/1000
23/23 - 0s - loss: 3.5655 - val_loss: 217.7626
Epoch 810/1000
23/23 - 0s - loss: 3.5085 - val_loss: 219.0260
Epoch 811/1000
23/23 - 

Epoch 928/1000
23/23 - 0s - loss: 5.4440 - val_loss: 206.1448
Epoch 929/1000
23/23 - 0s - loss: 4.7794 - val_loss: 195.3990
Epoch 930/1000
23/23 - 0s - loss: 7.8699 - val_loss: 189.6704
Epoch 931/1000
23/23 - 0s - loss: 5.5974 - val_loss: 223.0723
Epoch 932/1000
23/23 - 0s - loss: 4.0753 - val_loss: 205.6207
Epoch 933/1000
23/23 - 0s - loss: 3.8578 - val_loss: 198.9720
Epoch 934/1000
23/23 - 0s - loss: 3.4879 - val_loss: 200.4738
Epoch 935/1000
23/23 - 0s - loss: 3.9417 - val_loss: 205.5753
Epoch 936/1000
23/23 - 0s - loss: 3.4805 - val_loss: 200.0897
Epoch 937/1000
23/23 - 0s - loss: 3.9220 - val_loss: 212.1017
Epoch 938/1000
23/23 - 0s - loss: 4.7489 - val_loss: 213.6864
Epoch 939/1000
23/23 - 0s - loss: 5.3859 - val_loss: 200.5904
Epoch 940/1000
23/23 - 0s - loss: 4.3403 - val_loss: 197.6354
Epoch 941/1000
23/23 - 0s - loss: 4.0025 - val_loss: 220.8049
Epoch 942/1000
23/23 - 0s - loss: 3.8738 - val_loss: 203.2086
Epoch 943/1000
23/23 - 0s - loss: 3.5377 - val_loss: 205.6497
Epoch 94

<strong>You can refer to this [link](https://keras.io/models/sequential?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) to learn about other functions that you can use for prediction or evaluation.</strong>


Feel free to vary the following and note what impact each change has on the model's performance:

1.  Increase or decreate number of neurons in hidden layers
2.  Add more hidden layers
3.  Increase number of epochs


### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                                          |
| ----------------- | ------- | ---------- | ----------------------------------------------------------- |
| 2020-09-21        | 2.0     | Srishti    | Migrated Lab to Markdown and added to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>


This notebook is part of a course on **Coursera** called _Introduction to Deep Learning & Neural Networks with Keras_. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0101EN_Coursera_Week3_LAB1).


<hr>

Copyright © 2019 [IBM Developer Skills Network](https://cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DL0101EN-SkillsNetwork-20718188&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
